In [12]:
import tensorflow as tf

In [13]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [14]:
X = housing.data
y = housing.target
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

We are now using the keras functional API to build the model. The functional API is more flexible than the sequential API as it allows us to use non-linear topology for building the layers. For example in this notebook, we are using the Wide&Deep architecture in which some parts of the input are connected directly to the output allowing us to skip some connections.

In this example we are trying to send five features through the wide path and six features through the deep path.

In [15]:
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30,activation = "relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30,activation = "relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide,hidden2])
output = tf.keras.layers.Dense(1)(concat)
model = tf.keras.Model(inputs=[input_wide,input_deep],outputs = [output])

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
model.compile(loss = "mse", optimizer = optimizer, metrics = ["RootMeanSquaredError"])
X_train_wide, X_train_deep = X_train[:,:5] , X_train[:,2:]
X_valid_wide, X_valid_deep = X_valid[:,:5] , X_valid[:,2:]
X_test_wide, X_test_deep = X_test[:,:5], X_test[:,2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

In [19]:
history = model.fit((X_train_wide,X_train_deep),y_train,epochs = 20, validation_data = ((X_valid_wide,X_valid_deep),y_valid))
mse_test = model.evaluate((X_test_wide,X_test_deep),y_test)
y_pred = model.predict((X_new_wide,X_new_deep))

Epoch 1/20


/Users/anupamg/opt/miniconda3/envs/AI/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_9', 'keras_tensor_10']. Received: the structure of inputs=('*', '*')
  warnings.warn(


363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 489us/step - RootMeanSquaredError: 1.7714 - loss: 3.2553 - val_RootMeanSquaredError: 2.7305 - val_loss: 7.4556
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step - RootMeanSquaredError: 0.8024 - loss: 0.6445 - val_RootMeanSquaredError: 0.9808 - val_loss: 0.9620
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step - RootMeanSquaredError: 0.6927 - loss: 0.4800 - val_RootMeanSquaredError: 0.6853 - val_loss: 0.4697
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - RootMeanSquaredError: 0.6517 - loss: 0.4250 - val_RootMeanSquaredError: 0.6654 - val_loss: 0.4428
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - RootMeanSquaredError: 0.6355 - loss: 0.4039 - val_RootMeanSquaredError: 0.7392 - val_loss: 0.5465
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - RootMeanSquaredError: 0.6130 - loss: 0.3760 - val_RootMeanSquaredError: 0.6229 - val_loss: 0.3880
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step - RootMeanSquaredError: 0.6008 - 